#شبیه سازی مونت کارلو و کاهش واریانس
<div dir="rtl">
مدل بلک شولز بیان می کند که قیمت دارایی به عنوان تابعی از زمان$S(t)$ از یک حرکت هندسی براونی پیروی می کند:

$$dS=S\mu dt + S \sigma dW $$

یک راه جایگزین برای محاسبه قیمت اختیار خرید، استفاده از روش‌های مونت کارلو است. نقطه شروع، گرفتن معادله حرکت براونی برای سهام اصلی $S(t)$ است:

$$dS = S\,r dt +S\sigma dz\,,$$

جایی که ما $\mu =r$ را تنظیم کرده ایم، زیرا در یک محیط بدون ریسک کار می کنیم. این معادله را می توان با طرح اویلر گسسته کرد. با انجام این کار، سطح شاخص سهام $S_T$ را در $T$ آینده با سطح $S_t$ به صورت زیر بدست می آوریم:

$$S_{T} = S_{t}\exp\left(\left(r-\frac{1}{2}\sigma^{2}\right)(T-t) +\sigma \sqrt{T-t}\,z\right)\,,$$

که در آن $z$ یک متغیر تصادفی است که از توزیع نرمال پیروی می کند.
<div>


In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import random
from math import *
import time
import scipy.stats

In [ ]:
S0 = 100
K = 100 #ATM Call option
T = 1
sigma = 0.20
r = 0.01
mu = 0.15
n = 100 #Simulations number

<div dir="rtl">
n مسیر قیمت سهام را با حرکت هندسی براونی محاسبه کنید

In [ ]:
def GBM(S0,mu=0.15,sigma=0.3,T=1,n=100):
    BM = []
    steps = int(260*T) #Number of steps
    dt = T/steps #Time step
    
    tt = np.arange(0,T,dt) #Timesteps
    
    for sim in range(n): #Generates n paths
        S = np.zeros(steps)
        S[0] = S0
        for t in range(1,steps): #Simulates current Brownian motion
            drift = (mu - 0.5 * (sigma ** 2)) * dt
            dS = drift + sigma * np.random.normal(0,1) * np.sqrt(dt)
            S[t] = S[t-1] * np.exp(dS)
        BM.append(S)
    
    return BM
start_time = time.time()
BM = GBM(S0,r,sigma,T,n)
print("--- %s seconds --- " %(time.time() - start_time))
for B in BM:
    plt.plot(B)
plt.xlabel("$t$")
plt.ylabel("$S_{t}$")
plt.title("Stock Price Paths with Geometric Brownian Motion")
plt.show()

<div dir="rtl">

شبیه سازی مونت کارلو برای به دست آوردن قیمت اختیار خرید


In [ ]:
def MonteCarloEUCall(BM,K,r,T): #Monte Carlo for European Call options
    #BM List of stock prices paths 
    #K Strike 
    #r Risk free interest rate
    #T Maturity of the option
    call_payoffs = [max(B[-1] - K,0) for B in BM]        #Call payoffs for each stock price path
    mean_payoffs = np.mean(call_payoffs)
    return call_payoffs, mean_payoffs * np.exp(-r*T)     #payoffs , Discounted mean of the payoffs to get the option price
def empirical_sd(X):                                     #Computes the empirical standard deviation of a series
    n = len(X)
    s = 0
    mu = np.mean(X)
    for x in X:
        s += (x - mu)**2
    return np.sqrt(s/(n-1))
def confidence_interval(call_payoffs,alpha):
                                                          #call_payoffs List of the payoffs of the call options
                                                          #alpha Incertitude level
    
    z = scipy.stats.norm.ppf(1-alpha/2)
    mean_payoffs = np.mean(call_payoffs)
    CI = [np.round(mean_payoffs - z * (empirical_sd(call_payoffs)/np.sqrt(n)),4),
        np.round(mean_payoffs + z * ((empirical_sd(call_payoffs)/np.sqrt(n))),4)]
    CI = np.exp(-r*T) * np.array(CI)                      #Discount the values
    return CI
call_payoffs1, c1 = MonteCarloEUCall(BM,K,r,T)
alpha = 0.01
call_payoffs = [max(B[-1] - K,0) for B in BM]
CI1 = confidence_interval(call_payoffs,alpha)
print(f"The price of the call using the Monte Carlo Method is equal to : {round(c1,4)}")
print(f"The confidence interval of the call price at a {(1 - alpha)*100}% confidence level is {CI1}")
print("--- %s seconds --- " %(time.time() - start_time))

#کاهش واریانس
<div dir="rtl">
واریانس هر تخمین $\mathbb{E}[h(X)]$ به دست آمده را محاسبه کنید. بر اساس بحث در مورد روش های نمونه گیری در سخنرانی، که اصولاً انتظار می رود توزیع های پیشنهادی به کدام روش های نمونه گیری منجر شود. واریانس کمتر؟ نتایج شما چقدر با این انتظارات مطابقت دارد؟
سپس با استفاده از سه تکنیک مختلف کاهش واریانس قیمت گذاری می شود:

1.   متغیر متضاد
2.   متغیر کنترل



#متغیر متضاد


In [ ]:
def GBMAV(S0,mu=0.15,sigma=0.3,T=1,n=100):
    BM = []
    steps = int(260*T) #Number of steps
    dt = T/steps #Time step
    
    tt = np.arange(0,T,dt) #Timesteps
    
    for sim in range(n): #Generates n paths
        S = np.zeros(steps)
        S[0] = S0
        for t in range(1,steps): #Simulates current Brownian motion
            drift = (mu - 0.5 * (sigma ** 2)) * dt
            dS = drift + sigma * (1-np.random.normal(0,1)) * np.sqrt(dt)
            S[t] = S[t-1] * np.exp(dS)
        BM.append(S)
    
    return BM
start_time = time.time()
BMAV = GBMAV(S0,r,sigma,T,n)
print("--- %s seconds --- " %(time.time() - start_time))
for B in BMAV:
    plt.plot(B)
plt.xlabel("$t$")
plt.ylabel("$S_{t}$")
plt.title("Stock Price Paths with Geometric Brownian Motion")
plt.show()

#کنترل متغیر

In [ ]:
def BetaEstimation(X,Y):
    num = 0
    denom = 0
    e1,e2 =  np.mean(X), np.mean(Y)
    for i,x in enumerate(X):
        num += (X[i] - e1) * (Y[i] - e2)
        denom += (X[i] - e1) ** 2
    return num/denom
K = 100
Y = [max(B[-1] - K,0) for B in BM] 
X = [B[-1] for B in BM]
beta = BetaEstimation(X,Y) #Coefficient that minimizes the variance
Z = Y - beta * (X - BM[0][0]) #Control Variate #BM[0][0] = E[Z]
var1 = np.var(Y)
var2 = np.var(Z)
print(f"Original Variance of the call price : {var1}")
print(f"Reduced Variance of the call price : {var2}")
plt.plot(Z)